# Проект для «Викишоп» c BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

In [1]:
#!pip install catboost transformers

In [3]:
import gc
import re
import nltk
import string
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression

from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

import torch
from torch.utils.data import Dataset, DataLoader
from torch import cuda
import transformers
from tqdm import notebook, tqdm
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

import os.path
from scipy.stats import randint as sp_randint
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 29)

prjct_name = '13 Интернет магазин комментарии'

file_name = 'toxic_comments.csv'
snp_fname = 'inet_shp_commnts'
data_fldr= 'data'
snp_fldr = 'snapshots'
path_to_prjct = ['MyDrive', 'Data Science', 'Проекты', prjct_name]

local_path = os.path.join('G:' + os.sep, *map(''.join, path_to_prjct), data_fldr, file_name)
print(local_path)
web_path = os.path.join(os.sep, 'datasets', file_name)
print(web_path)
ggle_path = os.path.join(os.sep, 'content', 'drive', *map(''.join, path_to_prjct), data_fldr, file_name)
print(ggle_path)
snapsht_CB_path = os.path.join(os.sep, 'content', 'drive', *map(''.join, path_to_prjct), snp_fldr, snp_fname)
print(snapsht_CB_path)
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jovyan/nltk_data...


G:/MyDrive/Data Science/Проекты/13 Интернет магазин комментарии/data/toxic_comments.csv
/datasets/toxic_comments.csv
/content/drive/MyDrive/Data Science/Проекты/13 Интернет магазин комментарии/data/toxic_comments.csv
/content/drive/MyDrive/Data Science/Проекты/13 Интернет магазин комментарии/snapshots/inet_shp_commnts


## Подготовка

In [4]:
try:
    data_full  = pd.read_csv(web_path, index_col = 0)
except FileNotFoundError:
    try:
      data_full  = pd.read_csv(local_path, index_col = 0)
    except FileNotFoundError:
      data_full  = pd.read_csv(ggle_path, index_col = 0) 
print(f'\nОбзор данных датасета:', sep='')
display(data_full)
print(f'Общая информация о данных датасета ', file_name, ':', sep='')
data_full.info()
print(f'Количество дубликатов в данных датасета ', file_name, ':', sep='')
print(data_full.duplicated().sum(), '\n')
print(f'Колонки с пропущенными значениями в данных датасета ', file_name, ':\n', sep='')
print(data_full.isna().sum(), '\n')
print(f'Уникальные значения в колонке toxic датасета, распределение классов ', file_name, ':\n', sep='')
print(data_full['toxic'].value_counts(), '\n')


Обзор данных датасета:


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159446,""":::::And for the second time of asking, when ...",0
159447,You should be ashamed of yourself \n\nThat is ...,0
159448,"Spitzer \n\nUmm, theres no actual article for ...",0
159449,And it looks like it was actually you who put ...,0


Общая информация о данных датасета toxic_comments.csv:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB
Количество дубликатов в данных датасета toxic_comments.csv:
0 

Колонки с пропущенными значениями в данных датасета toxic_comments.csv:

text     0
toxic    0
dtype: int64 

Уникальные значения в колонке toxic датасета, распределение классов toxic_comments.csv:

0    143106
1     16186
Name: toxic, dtype: int64 



**Вывод** Данные загружены, пропуски отсуствуют, видим неравенство классов, что будем учитывать при обучении моделей.

Сделаем стратифицированную выборку из исходного датасета, в целях сокращения времени обработки данных, токенизации, подбора и обучения моделей.

In [5]:
data = data_full.groupby('toxic', group_keys=False).apply(lambda x: x.sample(frac=0.005, random_state=54321))
print(data['toxic'].value_counts())
stopwords = set(nltk_stopwords.words('english'))


0    716
1     81
Name: toxic, dtype: int64


Очистим память перед проведением обработки текстов датафрейма.

In [7]:
gc.collect()

81

Определим максимальную длину текстов и приведём их кодировку к Unicode.

In [8]:
def max_lenght(data):
    max_len = 0
    for i in data:
        if len(i) > max_len:
            max_len = len(i)
    return max_len
max_len = max_lenght(data['text'])
print(f'Максимальная длина текстовой записи в датафрейме: {max_len}')

Максимальная длина текстовой записи в датафрейме: 4887


Очистим тексты датафрейма от стоп-слов, лематизируем, заполним 0 исключенные значения, выровняв длину текстов

In [9]:
lemmatizer = WordNetLemmatizer()
def clear_text(text):
    paded_text =0
    
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = text.lower()#[w.lower() for w in text]
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    text = [word for word in text if word not in stopwords]        
    padded_text = text + ['0']*(max_len-len(text))
#    return text, paded_text
    return ' '.join(text), ' '.join(padded_text)
#    word_list = nltk.word_tokenize(text)
    #lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    #return lemmatized_output

data['text'] = data['text'].apply(lambda x: clear_text(x)[0])
data['paded_text'] = data['text'].apply(lambda x: clear_text(x)[1])

In [10]:
#print(data['text'][:4])

Используем для токенизации текстов BERT, сохранив также очищенные и уравненные по длине нулевыми значениями тексты в колонке 'paded_text'.

In [11]:
#tokenizer = transformers.AutoTokenizer.from_pretrained('bert-base-uncased')#, model_max_length=512)
tokenizer = transformers.AutoTokenizer.from_pretrained('unitary/toxic-bert', model_max_length=100)#, model_max_length=512)
#model = transformers.AutoModelForMaskedLM.from_pretrained('bert-base-uncased')
model = transformers.AutoModelForMaskedLM.from_pretrained('unitary/toxic-bert')

Some weights of the model checkpoint at unitary/toxic-bert were not used when initializing BertForMaskedLM: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized from the model checkpoint at unitary/toxic-bert and are newly initialized: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
You should probably TRAIN t

In [12]:
#tokenized = data['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, padding='max_length', truncation=True))
tokenized = data['paded_text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, padding='max_length', truncation=True))

In [13]:
padded = np.array([i for i in tokenized.values])
attention_mask = np.where(padded != 0, 1, 0)

In [14]:
torch.cuda.empty_cache()
gc.collect()

61

По рекоммендации из документации model.eval используется перед началом формировния обёрток, а model.train после.

In [15]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
embeddings = torch.Tensor().to(device)
model.eval()
batch_size = 26
batch_size = 1
embeddings = []
for i in notebook.tqdm(range((padded.shape[0] // batch_size))):
    batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]).cuda()
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch.cuda())
    embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

  0%|          | 0/798 [00:00<?, ?it/s]

In [16]:
model.train();

Объединяем массивы в единый массив, из которого создаём датафрейм Пандас. Делим данные на тренировочную и тестовую выборки. Задаём параметр стратификации в соотвествии с таргетом, чтобы не потерять соотношение классов.

In [17]:
features = pd.DataFrame(data=np.concatenate(embeddings), index=data)
target  = data['toxic']
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=54321)

## Обучение

LogisticRegression

Подбираем параметры модели из трёх словарей param_distributions, содержащих неконфликтующие параметры и определяем по максимально полученному значению f1 лучшие параметры для обучения.

In [ ]:
parameters = {'C': np.linspace(10, 20, num = 11, endpoint = True),
             'max_iter': [1000]}
lrm = LogisticRegression()
clf = GridSearchCV(lrm, parameters,
                  cv=5,
                  scoring='f1',
                  n_jobs=-1,
                  verbose=2,
                  error_score=0)
clf.fit(features_train, target_train)

Выводим параметры лучшей модели LogisticRegression.

In [ ]:
print(f"Наилучший показатель f1 на кросс-валидации : {clf.best_score_:.3f}")
print(f"Параметр регуляризации для лучшей модели: {clf.best_params_}")

CatBoostClassifier

In [22]:

#torch.cuda.empty_cache()
gc.collect()
model = CatBoostClassifier(eval_metric='F1', task_type='CPU', save_snapshot=True, snapshot_file=snapsht_CB_path)


# grid = {'learning_rate': [0.03, 0.1],
#         'depth': [4, 10],
#         'l2_leaf_reg': [1, 9]}

# randomized_search_result = model.randomized_search(grid,
#                                                    features_train,
#                                                    target_train,
#                                                    plot=True)
model_CB_best = model.fit(features_train, target_train)
#model_CB_best = randomized_search_result.best_estimator_
#model_CB_best.fit(features_train, target_train)

Learning rate set to 0.008503
0:	learn: 0.8333333	total: 8.95s	remaining: 2h 28m 58s
1:	learn: 0.8571429	total: 15.8s	remaining: 2h 11m 39s
2:	learn: 0.9059829	total: 24.5s	remaining: 2h 15m 49s
3:	learn: 0.9152542	total: 31.4s	remaining: 2h 10m 6s
4:	learn: 0.9000000	total: 38.1s	remaining: 2h 6m 12s
5:	learn: 0.9059829	total: 44.9s	remaining: 2h 4m 2s
6:	learn: 0.9059829	total: 51.8s	remaining: 2h 2m 31s
7:	learn: 0.8888889	total: 58.6s	remaining: 2h 1m 7s
8:	learn: 0.8793103	total: 1m 5s	remaining: 2h 6s
9:	learn: 0.8888889	total: 1m 12s	remaining: 1h 59m 4s
10:	learn: 0.8888889	total: 1m 19s	remaining: 1h 58m 23s
11:	learn: 0.8888889	total: 1m 25s	remaining: 1h 57m 39s
12:	learn: 0.9059829	total: 1m 32s	remaining: 1h 56m 56s
13:	learn: 0.8983051	total: 1m 39s	remaining: 1h 56m 22s
14:	learn: 0.9059829	total: 1m 45s	remaining: 1h 55m 52s
15:	learn: 0.9059829	total: 1m 52s	remaining: 1h 55m 25s
16:	learn: 0.9059829	total: 1m 59s	remaining: 1h 55m
17:	learn: 0.9059829	total: 2m 6s	rem

In [23]:
#print(model_CB_best.best_score_)
predicted = model_CB_best.predict(features_test)

print(f"Наилучший показатель f1 на кросс-валидации : {model_CB_best.get_best_score()}")
#print(f"Параметр регуляризации для лучшей модели: {model_CB_best.best_params_}")
# scores_CB = cross_val_score(model_CB_best, features_train, target_train, scoring='f1', cv=5)
# print(f'Параметры лучшей модели CatBoostClassifier по метрике F1: {abs(scores_CB.mean())}')

Наилучший показатель f1 на кросс-валидации : {'learn': {'Logloss': 0.0018953853472988506, 'F1': 1.0}}


In [24]:
predicted = model_CB_best.predict(features_test)
score = f1_score(target_test, predicted)
print(f"Наилучший показатель f1 на кросс-валидации тестовой выборке: {score}")

Наилучший показатель f1 на кросс-валидации тестовой выборке: 0.8648648648648648


## Выводы

В целях расширить опыт, в данном проекте импользовали нейронную модель лематизации, токенизации и эмбендинга - BERT. Она весьма упрощает этот процесс и по уверениям разработчиков делает это качественнее иных инструментов и способов. В связи с этим мы не стали готовить данные, проходя указанные отдельные этапы, а пропустили данные через алгоритм токенизации и эмбендинга и приступили к обучению и тестированию моделей кросс-валидацией. В качестве основной модели предсказаний использовали CatBoostClassifier, которая показала себя с наилучшей стороны по показателю метрик и времени обучения, чем обязана технологии градиентного бустинга, лежащей в её основе. 

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [X]  Значение метрики *F1* не меньше 0.75
- [X]  Выводы написаны